# Hilbert curve
- This example uses numpy's fft function to smoothen a path by applying a low-pass filter in the frequency domain.

In [1]:
if 'google.colab' in str(get_ipython()):
  try:
    import fullcontrol as fc
  except Exception as e:
    print (e)
    print('Attempting to install missing packages. Please wait ...')
    !pip install git+https://github.com/FullControlXYZ/fullcontrol --quiet
    import fullcontrol as fc
  from google.colab import files
from math import cos, tau

No module named 'fullcontrol'
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import numpy as np
def  hilbert(n):
  a=1+1j
  b=np.conj(a)
  z=np.array([0])
  for k in range(n):
    w=1j*np.conj(z)
    z=np.concatenate([w-a,z-b,z+a,b-w])/2.0
  return z
def moore(n):
  z=-np.conj(hilbert(n))
  w=np.concatenate([z+1+1j,z-1+1j])
  return np.concatenate([w,-1.0*w])/2.0
def area_length(x):
  deltax=np.roll(x,-1)-x
  area=sum(np.conj(x)*deltax).imag/2.0#the cross product
  length=sum(abs(deltax))
  return (area,length)
z4=moore(3)
a4,l4=area_length(z4)
r4=(a4/np.pi)**0.5

In [3]:
n=8
zu=np.concatenate([[z4[i-1]*(n-k)+z4[i]*k for k in range(1,n+1)] for i in range(len(z4))])/n
zu=np.roll(zu,-n//2+1)
zuf=np.fft.fft(zu)
rcos=(0.5*(1+np.cos(np.pi*2*np.array(list(range(len(zu))))/len(zu))))
rzu=r4*len(zu)
zuf[1]-=rzu
zuff=zuf*rcos**np.exp(2)
zuff[1]+=rzu
zk=np.fft.ifft(zuff)

In [34]:
# create design steps

# print part:
steps=[]
scale=20
n=len(zk)
nz=0.5
hl=0.2
j=0
z=0
t=np.linspace(0,2*np.pi,101)
pcirc=r4*scale*1.04*np.exp(1j*t)
z=hl
steps.append(fc.Extruder(on=False))

steps.extend(fc.travel_to(fc.Point(x=pcirc[0].real,y=pcirc[0].imag,z=z)))
steps.append(fc.Extruder(on=True))
steps.append(fc.ExtrusionGeometry(area_model='rectangle',width=1.0,height=hl))
for p in pcirc[1:]:
  steps.append(fc.Point(x=p.real,y=p.imag,z=z))
ew_old=-1
steps.extend(fc.travel_to(fc.Point(x=r4*scale,y=0*scale)))
steps.append(fc.Extruder(on=True))
z=0
while z<40+hl:
  z+=hl
  zuff=zuf*rcos**np.exp(10*((39-z)/20+0.1)**nz-1)
  zuff[1]+=rzu
  zk=np.fft.ifft(zuff)
  skip=1
  if z<15:
    skip=32
  elif z<20:
    skip=16
  elif z<25:
    skip=8
  elif z<30:
    skip=4
  elif z<35:
    skip=2
  else:
    skip=1
  ew=1-0.5*z/40
  steps.append(fc.ExtrusionGeometry(area_model='rectangle',width=ew,height=hl))
  for j,p in enumerate(zk):
    if j%skip==0:
      steps.append(fc.Point(x=p.real*scale,y=p.imag*scale,z=z))
print(len(steps))
# offset the whole procedure. z dictates the gap between the nozzle and the bed for the first layer, assuming the model was designed with a first layer z-position of 0
steps = fc.move(steps,fc.Point(x=100,y=100,z=0))


106359


In [35]:
# add annotations and plot
EW=0.5
EH=0.2
annotations = []
#annotations.append(fc.PlotAnnotation(point=steps[0], label = "Start"))
#annotations.append(fc.PlotAnnotation(point=steps[-1], label = "End"))
fc.transform(steps + annotations, 'plot', fc.PlotControls(color_type='print_sequence', style='line', initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))

Output hidden; open in https://colab.research.google.com to view.

In [36]:
#design parameters

design_name = 'hilbert_curve'
nozzle_temp = 220
bed_temp = 120
print_speed = 30*60
fan_percent = 0
EH = 0.2    # extrusion heigth
EW = 0.5    # extrusion width
printer_name = 'generic'
#printer_name = 'Prusa_Mendel'
gcode_controls = fc.GcodeControls(
    printer_name=printer_name,
    save_as=design_name,
    initialization_data={
        'primer': 'no_primer',
        'print_speed': print_speed,
        'nozzle_temp': nozzle_temp,
        'bed_temp': bed_temp,
        'fan_speed':fan_percent,
        'extrusion_width': EW,
        'extrusion_height': EH})
gcode = fc.transform(steps, 'gcode', gcode_controls)